In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn
sn.set_context("poster")

import torch
from torch import nn as nn
ttype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
ctype = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor
print(ttype)
import torch.nn.functional as F
from matplotlib import gridspec
from sithcon import SITHCon_Layer, _SITHCon_Core, iSITH

from tqdm.notebook import tqdm

import itertools
from csv import DictWriter
import os 
from os.path import join
import glob
ttype=torch.cuda.FloatTensor
import numpy as np
import pandas as pd
import pickle
from math import factorial
import random

# Experiment 3.A

In [ ]:
with open(join('data', 'trainX_norm.dill'), "rb") as f:
    trainX = pickle.load(f)
with open(join('data', 'trainY_norm.dill'), "rb") as f:
    trainY = pickle.load(f)


In [ ]:
SITH_perfs = None
for r in range(5):
    if SITH_perfs is None:
        SITH_perfs = pd.read_pickle(join("perf", "sithcon_hard_test_5032021_{}.dill".format(r)))
        SITH_perfs['run'] = [r]*SITH_perfs.shape[0]
    else:
        temp =  pd.read_pickle(join("perf", "sithcon_hard_test_5032021_{}.dill".format(r)))
        temp['run'] = [r]*temp.shape[0]
        SITH_perfs = SITH_perfs.append(temp)
SITH_perfs['adj_scale'] = 1./SITH_perfs.scal
print(SITH_perfs)
TCN_perfs = None
for r in range(5):
    if TCN_perfs is None:
        TCN_perfs = pd.read_pickle(join("perf", "tcn_hard_test_{}.dill".format(r)))
        TCN_perfs['run'] = [r]*TCN_perfs.shape[0]
    else:
        temp =  pd.read_pickle(join("perf", "tcn_hard_test_{}.dill".format(r)))
        temp['run'] = [r]*temp.shape[0]
        TCN_perfs = TCN_perfs.append(temp)
TCN_perfs['adj_scale'] = 1./TCN_perfs.scal
print(TCN_perfs)

In [ ]:
window_size = 20
lw=10
ms = 15
with sn.plotting_context("notebook", font_scale=3):
    fig = plt.figure(figsize=(30,10), )
    spec = gridspec.GridSpec(nrows=1, ncols=1, hspace=0.1, wspace=.5,
                             figure=fig)
    ax = fig.add_subplot(spec[0])
    Trainscale = 5
    ax.axvline(1.0, color='red', linestyle='--', linewidth=5)

    sn.lineplot(data=SITH_perfs, x='adj_scale', y='perf', marker='o', markersize=ms, 
            linewidth=lw, ax=ax)
    sn.lineplot(data=TCN_perfs, x='adj_scale', y='perf', marker='o', markersize=ms, 
            linewidth=lw, ax=ax)
    ax.set_xscale('log')
    ax.set_ylabel('Accuracy')
    ax.set_xlabel('Scale')
    ax.grid()
    ax.set_ylim(-0.05,1.05)

In [ ]:
window_size = 20
lw=5
ms = 15
with sn.plotting_context("notebook", font_scale=3):
    fig = plt.figure(figsize=(30,10), )
    spec = gridspec.GridSpec(nrows=1, ncols=1, hspace=0.1, wspace=.5,
                             figure=fig)
    ax = fig.add_subplot(spec[0])
    Trainscale = 5
    ax.axvline(1.0, color='red', linestyle='--', linewidth=5)

    sn.lineplot(data=SITH_perfs, x='adj_scale', y='perf', marker='o', markersize=10, 
            linewidth=5, ax=ax)
    sn.lineplot(data=TCN_perfs, x='adj_scale', y='perf', marker='o', markersize=10, 
            linewidth=5, ax=ax)

    ax.set_xscale('log')

    ax.set_ylim(-0.05,1.05)
    #plt.xticks([.1, .2, .4, .8, 1.0, 1.25, 2.5, 5.0, 10.], 
    #           [.1, .2, .4,"", 1.0, "", 2.5, 5.0, 10.])
    ax.set_xticks([.1,.2,.4,1.0,2.5,5.,10.])
    ax.set_xticklabels([.1,.2,.4,1.0,2.5,5.,10.], 
                      )
    ax.set_xlabel('Scale')
    ax.set_ylabel('Accuracy')
    ax.grid()
        
    ax = fig.add_subplot(spec[:2,5:11])
    ax.plot((sithdats.batch_idx*16+sithdats.epoch*25792)/25792, 
            sithdats.training_perf[:], linewidth=5)
    ax.plot((tcndats.batch_idx*16+tcndats.epoch*25792)/25792, 
            tcndats.training_perf, linewidth=5)
    ax.legend(['SITHCon', 'TCN'])
    ax.set_ylim(.7, 1.0)
    ax.set_ylabel("Training Accuracy")
    ax.set_xlabel("Epoch")
    ax.grid()
    ax.set_xlim(0, 10)
    
    iar = [3150, 3200, 3250, 3300, 3350, 3400, 3450, 3500, 3550, 3600]
    for x, idx in enumerate(iar):
        ax = fig.add_subplot(spec[x%5, int(x/5)*2:int(x/5)*2+2])
        ax.imshow(trainX[idx-3150], aspect='auto',interpolation='none')
        ax.set_yticks([])
        ax.set_xticks([])
        
        ax.set_ylabel(x)
        if int(x/5) > 0:
            ax.yaxis.set_label_position("right")
            ax.set_ylabel(x, rotation=0, labelpad=8, ha='left', va='center')
        else:
            ax.set_ylabel(x, rotation=0, labelpad=8, ha='right', va='center')
plt.savefig(join('figs', 'AudioMNIST_1.0_full.png'), dpi=150)
plt.savefig(join('figs', 'AudioMNIST_1.0_full.pdf'), dpi=150)
plt.savefig(join('figs', 'AudioMNIST_1.0_full.svg'), dpi=150)

# Experiment 4.2


In [ ]:
with open(join('data', 'trainX_sawtooth.dill'), "rb") as f:
    trainX = pickle.load(f)
with open(join('data', 'trainY_sawtooth.dill'), "rb") as f:
    trainY = pickle.load(f)


In [ ]:
TCN_perfs = pd.read_pickle(join("perf", "TCN_mixed_test_5052021_sawtooth_0.dill"))
#SITH_perfs = pd.read_pickle(join("perf", "sith_mixed_test_5052021_all_0.dill"))
SITH_perfs = pd.read_pickle(join("perf", "sith_mixed_test_startsat1_5032021_sawtooth.dill"))
sithdats = pd.read_csv(join('..', 'AudioMedium','perf','sithcon_audiomixed_startsat1_5052021_sawtooth_0.csv'))
tcndats = pd.read_csv(join('..', 'AudioMedium','perf','TCN_audiomixed_scalable_5052021_sawtooth_0.csv'))


In [ ]:
window_size = 20
lw=5
ms = 15
with sn.plotting_context("notebook", font_scale=3):
    fig = plt.figure(figsize=(30,10), )
    spec = gridspec.GridSpec(nrows=5, ncols=16, hspace=.1, wspace=.15,
                             figure=fig)
    ax = fig.add_subplot(spec[3:,4:10])
    Trainscale = 5
    ax.axvline(.1, color='red', linestyle='--', linewidth=5)
    ax.axvline(.4, color='red', linestyle='--', linewidth=5)
    ax.axvline(1.0, color='red', linestyle='--', linewidth=5)
    ax.axvline(2.5, color='red', linestyle='--', linewidth=5)
    ax.axvline(10, color='red', linestyle='--', linewidth=5)
    ax.plot(1/SITH_perfs.scal, SITH_perfs.perf, marker='o', markersize=10, 
            linewidth=5)
    ax.plot(1/TCN_perfs.scal, TCN_perfs.perf, marker='o', markersize=10, 
            linewidth=5)

    ax.set_xscale('log')

    ax.set_ylim(-0.05,1.05)
    #plt.xticks([.1, .2, .4, .8, 1.0, 1.25, 2.5, 5.0, 10.], 
    #           [.1, .2, .4,"", 1.0, "", 2.5, 5.0, 10.])
    ax.set_xticks([.1,.2,.4,1.0,2.5,5.,10.])
    ax.set_xticklabels([.1,.2,.4,1.0,2.5,5.,10.], 
                      )
    ax.set_xlabel('Scale')
    ax.set_ylabel('Accuracy')
    ax.grid()
        
    ax = fig.add_subplot(spec[:2,4:10])
    ax.plot((sithdats.batch_idx*16+sithdats.epoch*25792)/25792, 
            sithdats.training_perf[:], linewidth=5)
    ax.plot((tcndats.batch_idx*16+tcndats.epoch*25792)/25792, 
            tcndats.training_perf, linewidth=5)
    ax.legend(['SITHCon', 'TCN'])
    ax.set_ylim(.7, 1.0)
    ax.set_ylabel("Training Accuracy")
    ax.set_xlabel("Epoch")
    ax.grid()
    ax.set_xlim(0, 10)
    
    iar = [320, 7970, 11570, 16520, 26420]
    scal = [10., 2.5, 1.0, .4, .1]
    for x, idx in enumerate(iar):
        ax = fig.add_subplot(spec[x:x+1, :3])
        ax.imshow(trainX[idx], aspect='auto',interpolation='none')
        ax.set_yticks([])
        ax.set_xticks([])
        ax.set_ylabel(scal[x])
plt.savefig(join('figs', 'AudioMNIST_Mixed_full.png'), dpi=150)
plt.savefig(join('figs', 'AudioMNIST_Mixed_full.pdf'), dpi=150)
plt.savefig(join('figs', 'AudioMNIST_Mixed_full.svg'), dpi=150)